In [30]:
import requests
import pandas as pd
from time import sleep
class CkanConsumer:
    def __init__(self, main_url:str, secure:bool = True):
        if secure:
            self.url = f'https://{main_url}/api/action/datastore_search'
        else:
            self.url = f'http://{main_url}/api/action/datastore_search'

    def request(self, resource_id) -> pd.DataFrame:
        params = {
            'limit': 1000,
            'offset': 0,
            'resource_id': resource_id
        }
        data = []

        print(f'[INFO] - Getting data from {self.url} on resource {resource_id} at offset {len(data)}')
        response = requests.get(self.url, params=params)
        total = response.json()['result']['total']
        data.extend(response.json()['result']['records'])

        while len(data) < total:
            params['offset'] = len(data)
            print(f'[INFO] - Getting data from {self.url} on resource {resource_id} at offset {len(data)}')
            response = requests.get(self.url, params=params)
            while response.status_code != 200:
                print(f'[ERROR] - Response code {response.status_code} from {self.url} on resource {resource_id} at offset {len(data)}')
                print(f'[INFO] - Waiting 5 seconds before trying again')
                sleep(5)
                response = requests.get(self.url, params=params)
            data.extend(response.json()['result']['records'])
        
        print(f'[INFO] - Total of {len(data)} records retrieved from {self.url} on resource {resource_id}')
        #Reassembling the dataframe
        df = pd.DataFrame(data)
        return df

In [64]:
ufpi_consumer = CkanConsumer('dados.ufpi.br', secure=True) 
cursos_ufpi = ufpi_consumer.request('afb0de00-8f39-4f9c-9197-4364f7312a98')
docentes_ufpi = ufpi_consumer.request('a34d7d7e-30af-41f0-81cf-cd10b6f078bd')
discentes_ufpi = ufpi_consumer.request('00658685-cd1b-47f4-9f42-107c45a1e4e1')


[INFO] - Getting data from https://dados.ufpi.br/api/action/datastore_search on resource afb0de00-8f39-4f9c-9197-4364f7312a98 at offset 0
[INFO] - Total of 252 records retrieved from https://dados.ufpi.br/api/action/datastore_search on resource afb0de00-8f39-4f9c-9197-4364f7312a98
[INFO] - Getting data from https://dados.ufpi.br/api/action/datastore_search on resource a34d7d7e-30af-41f0-81cf-cd10b6f078bd at offset 0
[INFO] - Getting data from https://dados.ufpi.br/api/action/datastore_search on resource a34d7d7e-30af-41f0-81cf-cd10b6f078bd at offset 1000
[INFO] - Total of 1812 records retrieved from https://dados.ufpi.br/api/action/datastore_search on resource a34d7d7e-30af-41f0-81cf-cd10b6f078bd
[INFO] - Getting data from https://dados.ufpi.br/api/action/datastore_search on resource 00658685-cd1b-47f4-9f42-107c45a1e4e1 at offset 0
[INFO] - Getting data from https://dados.ufpi.br/api/action/datastore_search on resource 00658685-cd1b-47f4-9f42-107c45a1e4e1 at offset 1000
[INFO] - Gettin

In [67]:
discentes_ufpi.head()

,nome,curso,sexo,nome_unidade,matricula,_id
0,LUCIELE DAMAS PEREIRA,ENGENHARIA CARTOGRÁFICA E DE AGRIMENSURA,F,CENTRO DE TECNOLOGIA,20199035013,1
1,VINICIUS SOARES DE FREITAS,DIREITO,M,CENTRO DE CIENCIAS HUMANAS E LETRAS,20199005060,2
2,MARCIA CRISTINA FAUSTINO HOLANDA,LETRAS - LÍNGUA PORTUGUESA E LITERATURA DE LÍN...,F,CENTRO DE CIENCIAS HUMANAS E LETRAS,20219033827,3
3,ROSANE MARIA DAMASCENO DE SOUSA MIRANDA,ADMINISTRAÇÃO PÚBLICA,M,CENTRO DE EDUCACAO ABERTA E A DISTANCIA,20109158681,4
4,MARIA EDUARDA DO NASCIMENTO SOUSA,DESIGN DA MODA E ESTILISMO,F,CENTRO DE CIENCIAS DA EDUCACAO,20219017547,5
